In [ ]:
import pandahouse as ph
from airflow import DAG
from airflow.operators.python_operator import PythonOperator # Так как мы пишет такси в питоне
from datetime import datetime,  timedelta
from airflow.decorators import dag, task
from airflow.operators.python import get_current_context 
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
from datetime import datetime,  timedelta, date
from matplotlib.gridspec import GridSpec


schedule_interval = '50 10 * * *' # cron - выражение. Каждый день в 10:50 утра

# дефолтные аргументы для DAG
default_args = {
    'owner': 'r_muksinov', # владелец
    'depends_on_past': False, # не зависит от  успешности прошлого запуска 
    'retries': 2, # количество рестартов
    'retry_delay': timedelta(minutes=5), # пауза между рестартами
    'start_date': datetime(2023, 4, 9) # начало выполнения
}



# подключение к БД
connection = {'host': '******',
              'database': '******',
              'user': '******', 
              'password': '******'
              }

  
chat_id = ******
my_token = '*******' 
bot = telegram.Bot(token=my_token) # получаем доступ

@dag(default_args=default_args, schedule_interval=schedule_interval, catchup=False, tags=['r.muksinov'])
def muksinov_united_metrics_bot():
    @task()
    def query_message_0():
        query0 = '''
                 SELECT post_id, countIf(action='like') likes, countIf(action='view') views, countIf(action='like') / countIf(action='view') CTR
                 FROM simulator_20230320.feed_actions
                 WHERE toDate(time) = yesterday() - 1
                 GROUP BY post_id
                 ORDER BY CTR DESC
                 LIMIT 10
                 '''
        act = ph.read_clickhouse(query0, connection=connection)
        today = date.today()
        msg = f'Топовые посты по CTR за вчерашний ({today - timedelta(days = 1)}) день:\n\n{act.to_markdown()}'
        
        return msg
           
    @task()
    def query_1_retention():
        # запрос для ретеншн рейт
        query1 = '''
                SELECT * 
                FROM
                (SELECT act_date, first_date, active_users,
                        rank() over (PARTITION BY first_date ORDER BY act_date) as lifetime
                FROM

                (SELECT toString(act_date) AS act_date, toString(first_date) AS first_date, COUNT(user_id) AS active_users
                FROM 
                    (SELECT user_id, min(toDate(time)) as first_date
                    FROM simulator_20230320.feed_actions
                    GROUP BY user_id
                    HAVING first_date > today() - 15
                    ) AS  cohort_date

                    JOIN 

                    (SELECT DISTINCT user_id, toDate(time) AS act_date
                    FROM simulator_20230320.feed_actions
                    WHERE source = 'organic') AS user_active

                USING user_id
                GROUP BY act_date, first_date
                ORDER BY first_date, act_date) tbl) tb2 
                '''
        retention = ph.read_clickhouse(query1, connection=connection)
        
        max_active = retention.groupby('first_date', as_index=False)['active_users'].max()
        retention = retention.merge(max_active, how='inner', on='first_date')
        retention['normalise'] = round((retention['active_users_x'] / retention['active_users_y']), 3)
        retention = retention.pivot_table(index='first_date', columns='lifetime', values='normalise', aggfunc='max')
        
        return  retention

    @task()
    def query_2_DAUfeed():    
        # запрос DAU по новостной ленте за последний месяц
        query2 = '''
                SELECT toDate(time) as date, count(distinct user_id) unique_users
                FROM simulator_20230320.feed_actions
                WHERE time > today() - 30
                GROUP BY date
                '''
        DAU_feed = ph.read_clickhouse(query2, connection=connection)
        return DAU_feed
    
    @task()
    def query_3_DAUmsg():    
        # запрос DAU по мессенджеру за последний месяц
        query3 = '''
                SELECT toDate(time) as date, count(distinct user_id) unique_users
                FROM simulator_20230320.message_actions
                WHERE time > today() - 30
                GROUP BY date
                '''
        DAU_msg = ph.read_clickhouse(query3, connection=connection)
        return DAU_msg
    
    @task()
    def query_4_sticky():   
        # запрос для sticky factor
        query4 = '''
                SELECT day, round((user_qty/MAU * 100), 2) AS stiky_factor
                FROM 
                (SELECT toDate(time) AS day, count(distinct user_id) user_qty
                FROM simulator_20230320.feed_actions
                WHERE time > today() - 30
                GROUP BY day
                ORDER BY day) DAU,
                (SELECT count(distinct user_id) MAU
                FROM simulator_20230320.feed_actions
                WHERE time > today() - 30
                GROUP BY toYYYYMM(time)) AS M
                '''
        sticky = ph.read_clickhouse(query4, connection=connection) 
        return sticky
    
    @task()
    def query_5_CTR(): 
        # запрос для CTR 
        query5 = '''
                SELECT toDate(time) day,                            
                countIf(action='like') / countIf(action='view') CTR
                FROM simulator_20230320.feed_actions
                WHERE time > today() - 30
                GROUP BY day
                '''
        CTR = ph.read_clickhouse(query5, connection=connection)
        return CTR

    @task()
    def query_6_useractivity():
        #запрос для пользовталеской активности 
        query6 = '''
                SELECT *
                FROM
                (SELECT current_week, status, CAST(COUNT(DISTINCT user_id) as int)  as user_qty
                FROM
                (SELECT user_id, groupUniqArray(toStartOfWeek(time, 1)) week_visits, 
                arrayJoin(week_visits) current_week,
                CASE 
                  WHEN has(week_visits, date_add(week, -1, current_week)) = 1 THEN 'retained' ELSE 'new'
                END AS status
                FROM simulator_20230320.feed_actions
                GROUP BY user_id
                ORDER BY user_id) tab1
                GROUP BY current_week, status
                ORDER BY current_week

                UNION ALL

                SELECT current_week, status, CAST(COUNT(DISTINCT user_id) as int) *-1 as user_qty
                FROM
                (SELECT user_id, groupUniqArray(toStartOfWeek(time, 1)) week_visits, 
                date_add(week, +1, arrayJoin(week_visits)) current_week,
                CASE 
                  WHEN has(week_visits, current_week) = 1 THEN 'retained' ELSE 'gone'
                END AS status
                FROM simulator_20230320.feed_actions
                GROUP BY user_id
                HAVING status != 'retained'
                ORDER BY user_id) tab2
                GROUP BY current_week, status
                ORDER BY current_week DESC, status) TAB
                WHERE current_week != date_add(week, +1, toStartOfWeek(today(), 1))
                '''
        user_activity = ph.read_clickhouse(query6, connection=connection)
        user_activity = user_activity.pivot_table(index='current_week', columns='status', values='user_qty', aggfunc='sum').reset_index()
        user_activity['current_week'] = user_activity['current_week'].astype({'current_week': 'string'})
        return user_activity
    
    @task()
    def query_7_sources():    
        # запрос для источников пользователей в ленте 
        query7 = '''
                 SELECT toDate(time) date, uniq(user_id) users, source
                 FROM simulator_20230320.feed_actions
                 WHERE time > today() - 30 
                 GROUP BY date, source
                 ORDER BY date
                 '''
        user_source = ph.read_clickhouse(query7, connection=connection)
        return user_source

    @task()
    def visualise(retention, DAU_feed, DAU_msg, sticky, CTR, user_activity, user_source, message):
        fig = plt.figure(figsize=(26, 42))
        axes = GridSpec(5, 4, figure=fig)
        sns.set(font_scale=1.8)
        # retention rate
        axes1 = fig.add_subplot(axes[0, :])
        sns.heatmap(data=retention, annot=True, annot_kws={"size": 18}, cmap=sns.cubehelix_palette(as_cmap=True), fmt='g', cbar=False, ax=axes1)
        axes1.set_title('retention rate', fontsize=24)
        axes1.set_ylabel('когорты по датам', fontsize=22)
        axes1.set_xlabel('лайфтайм', fontsize=18)
        axes1.tick_params(axis='x', labelsize=18)
        

        # DAU по новостной ленте
        axes2 = fig.add_subplot(axes[1, :-2])
        sns.lineplot(data=DAU_feed.pivot_table(index='date'), ax=axes2)
        axes2.set_title('DAU по новостной ленте', fontsize=24)
        axes2.set_ylabel('количество пользователей', fontsize=22)
        axes2.set_xlabel('', fontsize=18)
        axes2.tick_params(axis='x', labelsize=18, rotation=90)
        
        # DAU по мессенджеру
        axes3 = fig.add_subplot(axes[1, 2:])
        sns.lineplot(data=DAU_msg.pivot_table(index='date'), ax=axes3)
        axes3.set_title('DAU по мессенджеру', fontsize=24)
        axes3.set_ylabel('количество пользователей', fontsize=22)
        axes3.set_xlabel('', fontsize=18)
        axes3.tick_params(axis='x', labelsize=18, rotation=90)
        
    
        # Sticky factor
        axes4 = fig.add_subplot(axes[2, :-2])
        sns.lineplot(data=sticky.pivot_table(index='day'), ax=axes4)
        axes4.set_title('Sticky factor', fontsize=24)
        axes4.set_ylabel('%', fontsize=22)
        axes4.set_xlabel('', fontsize=18)
        axes4.tick_params(axis='x', labelsize=18, rotation=90)
        
        # CTR
        axes4 = fig.add_subplot(axes[2, 2:])
        sns.lineplot(data=CTR.pivot_table(index='day'), ax=axes4)
        axes4.set_title('CTR', fontsize=24)
        axes4.set_ylabel('%', fontsize=22)
        axes4.set_xlabel('', fontsize=18)
        axes4.tick_params(axis='x', labelsize=18, rotation=90)
        
 
        # приток и отток аудитории по неделям
        axes4 = fig.add_subplot(axes[3, :])
        user_activity.plot(kind='bar', x='current_week', stacked=True, ax=axes4)
        axes4.set_title('Аудитория по неделям', fontsize=24)
        axes4.set_ylabel('количество пользователей', fontsize=22)
        axes4.set_xlabel('', fontsize=18)
        axes4.tick_params(axis='x', labelsize=18)


        # аудитория по источникам привлечения
        axes5 = fig.add_subplot(axes[4, :])
        sns.lineplot(data=user_source, x='date', y='users', hue='source', ax=axes5)
        axes5.set_title('Аудитория по источникам првлечения', fontsize=24)
        axes5.set_ylabel('количество пользователей', fontsize=22)
        axes5.set_xlabel('', fontsize=18)
        axes5.tick_params(axis='x', labelsize=18, rotation=90)
        
        plt.tight_layout()
        plot_object = io.BytesIO()
        plt.savefig(plot_object, dpi=100)
        plot_object.seek(0)
        plot_object.name = 'metrics.png'
        plt.close()
        bot.sendPhoto(chat_id=chat_id, photo=plot_object, caption=message)
      
    message = query_message_0() 
    retention = query_1_retention()
    DAU_feed = query_2_DAUfeed()
    DAU_msg = query_3_DAUmsg()
    sticky = query_4_sticky()
    CTR = query_5_CTR()
    user_activity = query_6_useractivity()
    user_source = query_7_sources()    
    
    visualise(retention, DAU_feed, DAU_msg, sticky, CTR, user_activity, user_source, message)
    
muksinov_united_metrics_bot = muksinov_united_metrics_bot()